In [34]:
!pip install --quiet optuna

In [35]:
import pandas as pd
import tensorflow as tf
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split
import optuna

In [36]:
data = pd.read_csv('./sentences.csv')
data.head()

,sentence_number,word,tag
0,1,como,HOW
1,1,usar,USE
2,1,lucros,PROFIT
3,1,e,CONJ
4,1,reinvestir,INVEST


In [37]:
vocab = set(data['word'])
vocab_list = list(vocab)

word2idx = {}
idx2word = {}

for idx, word in enumerate(vocab_list):
    word2idx[word] = idx
    idx2word[idx] = word

tags = set(data['tag'])
tags_list = list(tags)

tag2idx = {}
idx2tag = {}

for idx, tag in enumerate(tags_list):
    tag2idx[tag] = idx
    idx2tag[idx] = tag

In [38]:
converted_data = data.copy()

converted_data['word'] = converted_data['word'].transform(lambda word: word2idx[word])
converted_data['tag'] = converted_data['tag'].transform(lambda tag: tag2idx[tag])
converted_data.head()

,sentence_number,word,tag
0,1,103,3
1,1,42,22
2,1,65,70
3,1,22,34
4,1,113,28


In [39]:
list_data = converted_data.groupby(['sentence_number'])['word', 'tag'].agg(lambda i: list(i))
list_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,word,tag
sentence_number,,
1,"[103, 42, 65, 22, 113, 44, 66, 80, 11, 54]","[3, 22, 70, 34, 28, 56, 54, 16, 31, 52]"
2,"[103, 39, 17, 15, 5, 122, 53, 117, 73, 12, 16,...","[3, 65, 12, 33, 24, 44, 40, 67, 6, 48, 54, 12,..."
3,"[121, 30, 104, 83, 114, 111]","[41, 41, 47, 25, 25, 25]"
4,"[30, 50, 19, 78, 108, 88, 87, 122]","[41, 5, 26, 68, 41, 29, 41, 44]"
5,"[92, 16, 108, 37, 2, 112, 79, 102, 51, 68, 89]","[11, 54, 41, 12, 0, 24, 64, 50, 28, 24, 25]"


In [40]:
[idx2word[idx] for idx in list_data['word'][5]]

['quais',
 'são',
 'os',
 'melhores',
 'recursos',
 'para',
 'aprender',
 'sobre',
 'investimento',
 'em',
 'títulos']

In [41]:
max_len = list_data['word'].map(len).max()
max_len

28

In [42]:
pad_tokens = tf.keras.preprocessing.sequence.pad_sequences(list_data['word'])
pad_tags = tf.keras.preprocessing.sequence.pad_sequences(list_data['tag'])
pad_tags = [tf.keras.utils.to_categorical(tag, num_classes=len(tags)) for tag in pad_tags]

train_tokens, test_tokens, train_tags, test_tags = train_test_split(pad_tokens, pad_tags)

In [43]:
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers

In [44]:
input_dim = len(vocab)
input_length = max_len

def objective(trial):
    embedding_output_dim = trial.suggest_int("embedding_output_dim", 16, 256)

    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=embedding_output_dim, input_length=input_length),
        Bidirectional(LSTM(units=embedding_output_dim, return_sequences=True), merge_mode='concat'),
        LSTM(units=embedding_output_dim, return_sequences=True),
        TimeDistributed(Dense(len(tags), activation='relu'))
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    num_epochs = trial.suggest_int("num_epochs", 5, 150)

    model.fit(train_tokens, np.array(train_tags), verbose=1, epochs=num_epochs)

    return model.evaluate(test_tokens, np.array(test_tags), return_dict=True)['accuracy']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=6)

[I 2021-08-15 15:21:07,878] A new study created in memory with name: no-name-b1b23e11-6489-4631-bde6-a29e50c5cc6b


Epoch 1/82
1/1 [==============================] - 4s 4s/step - loss: 6.3831 - accuracy: 0.0020
Epoch 2/82
1/1 [==============================] - 0s 277ms/step - loss: 3.7500 - accuracy: 0.6310
Epoch 3/82
1/1 [==============================] - 0s 267ms/step - loss: 3.4194 - accuracy: 0.6310
Epoch 4/82
1/1 [==============================] - 0s 275ms/step - loss: 3.3275 - accuracy: 0.6329
Epoch 5/82
1/1 [==============================] - 0s 286ms/step - loss: 3.2429 - accuracy: 0.6329
Epoch 6/82
1/1 [==============================] - 0s 319ms/step - loss: 3.2320 - accuracy: 0.6329
Epoch 7/82
1/1 [==============================] - 0s 307ms/step - loss: 3.2326 - accuracy: 0.6329
Epoch 8/82
1/1 [==============================] - 0s 312ms/step - loss: 3.1916 - accuracy: 0.6310
Epoch 9/82
1/1 [==============================] - 0s 315ms/step - loss: 3.1466 - accuracy: 0.6329
Epoch 10/82
1/1 [==============================] - 0s 312ms/step - loss: 3.1022 - accuracy: 0.6329
Epoch 11/82
1/1 [=====

[I 2021-08-15 15:21:40,247] Trial 0 finished with value: 0.6734693646430969 and parameters: {'embedding_output_dim': 243, 'num_epochs': 82}. Best is trial 0 with value: 0.6734693646430969.


Epoch 1/64
1/1 [==============================] - 4s 4s/step - loss: 11.2578 - accuracy: 0.0099
Epoch 2/64
1/1 [==============================] - 0s 33ms/step - loss: 4.8734 - accuracy: 0.5714
Epoch 3/64
1/1 [==============================] - 0s 38ms/step - loss: 4.4257 - accuracy: 0.6310
Epoch 4/64
1/1 [==============================] - 0s 34ms/step - loss: 4.2057 - accuracy: 0.6310
Epoch 5/64
1/1 [==============================] - 0s 34ms/step - loss: 4.0935 - accuracy: 0.6310
Epoch 6/64
1/1 [==============================] - 0s 40ms/step - loss: 4.0386 - accuracy: 0.6310
Epoch 7/64
1/1 [==============================] - 0s 44ms/step - loss: 4.0393 - accuracy: 0.6310
Epoch 8/64
1/1 [==============================] - 0s 39ms/step - loss: 3.9859 - accuracy: 0.6310
Epoch 9/64
1/1 [==============================] - 0s 40ms/step - loss: 3.9401 - accuracy: 0.6310
Epoch 10/64
1/1 [==============================] - 0s 41ms/step - loss: 3.8930 - accuracy: 0.6310
Epoch 11/64
1/1 [=============

[I 2021-08-15 15:21:48,578] Trial 1 finished with value: 0.6326530575752258 and parameters: {'embedding_output_dim': 54, 'num_epochs': 64}. Best is trial 0 with value: 0.6734693646430969.


...

[I 2021-08-15 15:22:09,985] Trial 3 finished with value: 0.6479591727256775 and parameters: {'embedding_output_dim': 87, 'num_epochs': 95}. Best is trial 0 with value: 0.6734693646430969.


Epoch 1/72
1/1 [==============================] - 4s 4s/step - loss: 7.2894 - accuracy: 0.0000e+00
Epoch 2/72
1/1 [==============================] - 0s 84ms/step - loss: 4.2485 - accuracy: 0.6310
Epoch 3/72
1/1 [==============================] - 0s 80ms/step - loss: 3.8303 - accuracy: 0.6310
Epoch 4/72
1/1 [==============================] - 0s 80ms/step - loss: 3.6241 - accuracy: 0.6310
Epoch 5/72
1/1 [==============================] - 0s 84ms/step - loss: 3.4395 - accuracy: 0.6310
Epoch 6/72
1/1 [==============================] - 0s 77ms/step - loss: 3.2506 - accuracy: 0.6310
Epoch 7/72
1/1 [==============================] - 0s 86ms/step - loss: 3.1766 - accuracy: 0.6310
Epoch 8/72
1/1 [==============================] - 0s 86ms/step - loss: 3.0999 - accuracy: 0.6310
Epoch 9/72
1/1 [==============================] - 0s 86ms/step - loss: 3.0350 - accuracy: 0.6310
Epoch 10/72
1/1 [==============================] - 0s 87ms/step - loss: 3.0112 - accuracy: 0.6310
Epoch 11/72
1/1 [==========

[I 2021-08-15 15:22:21,506] Trial 4 finished with value: 0.6530612111091614 and parameters: {'embedding_output_dim': 112, 'num_epochs': 72}. Best is trial 0 with value: 0.6734693646430969.


Epoch 1/82
1/1 [==============================] - 4s 4s/step - loss: 14.1481 - accuracy: 0.0040
Epoch 2/82
1/1 [==============================] - 0s 113ms/step - loss: 13.5462 - accuracy: 0.0377
Epoch 3/82
1/1 [==============================] - 0s 106ms/step - loss: 5.0059 - accuracy: 0.0417
Epoch 4/82
1/1 [==============================] - 0s 111ms/step - loss: 4.3944 - accuracy: 0.5179
Epoch 5/82
1/1 [==============================] - 0s 113ms/step - loss: 4.1754 - accuracy: 0.6310
Epoch 6/82
1/1 [==============================] - 0s 109ms/step - loss: 3.9785 - accuracy: 0.6310
Epoch 7/82
1/1 [==============================] - 0s 106ms/step - loss: 3.8582 - accuracy: 0.6310
Epoch 8/82
1/1 [==============================] - 0s 127ms/step - loss: 3.7918 - accuracy: 0.6310
Epoch 9/82
1/1 [==============================] - 0s 115ms/step - loss: 3.7743 - accuracy: 0.6310
Epoch 10/82
1/1 [==============================] - 0s 120ms/step - loss: 3.7507 - accuracy: 0.6310
Epoch 11/82
1/1 [===

[I 2021-08-15 15:22:37,668] Trial 5 finished with value: 0.6581632494926453 and parameters: {'embedding_output_dim': 139, 'num_epochs': 82}. Best is trial 0 with value: 0.6734693646430969.


In [51]:
input_dim = len(vocab)
input_length = max_len

best_trial_params = study.best_trial.params

embedding_output_dim = best_trial_params['embedding_output_dim']
#num_epochs = best_trial_params['num_epochs']
num_epochs=30

model = Sequential([
    Embedding(input_dim=input_dim, output_dim=embedding_output_dim, input_length=input_length),
    Bidirectional(LSTM(units=embedding_output_dim, return_sequences=True, dropout=0.01), merge_mode='concat'),
    LSTM(units=embedding_output_dim, return_sequences=True, dropout=0.01),
    TimeDistributed(Dense(len(tags), activation='relu'))
])

optimizer = optimizers.Adam(clipvalue=0.5)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(train_tokens, np.array(train_tags), epochs=num_epochs)

model.evaluate(test_tokens, np.array(test_tags))

Epoch 1/30
1/1 [==============================] - 4s 4s/step - loss: 7.1426 - accuracy: 0.0020
Epoch 2/30
1/1 [==============================] - 0s 241ms/step - loss: 4.9301 - accuracy: 0.6329
Epoch 3/30
1/1 [==============================] - 0s 247ms/step - loss: 4.0679 - accuracy: 0.6310
Epoch 4/30
1/1 [==============================] - 0s 242ms/step - loss: 3.6341 - accuracy: 0.6310
Epoch 5/30
1/1 [==============================] - 0s 254ms/step - loss: 3.5273 - accuracy: 0.6508
Epoch 6/30
1/1 [==============================] - 0s 250ms/step - loss: 3.5008 - accuracy: 0.6667
Epoch 7/30
1/1 [==============================] - 0s 245ms/step - loss: 3.4551 - accuracy: 0.6706
Epoch 8/30
1/1 [==============================] - 0s 245ms/step - loss: 3.2719 - accuracy: 0.6706
Epoch 9/30
1/1 [==============================] - 0s 248ms/step - loss: 3.0479 - accuracy: 0.6488
Epoch 10/30
1/1 [==============================] - 0s 250ms/step - loss: 2.8707 - accuracy: 0.6409
Epoch 11/30
1/1 [=====

[3.474992513656616, 0.6326530575752258]

In [52]:
t = 'como usar lucros e reinvestir sem ser esmagado por impostos'.split(' ')
t2 = [word2idx[word] for word in t if word in vocab]
t3 = model.predict(t2)
[idx2tag[np.argmax(cat)] for cat in t3]

['RESOURCES',
 'RESOURCES',
 'RESOURCES',
 'RESOURCES',
 'RESOURCES',
 'RESOURCES',
 'RESOURCES',
 'RESOURCES',
 'RESOURCES',
 'RESOURCES']